#  Lottery Notebook

### imports :

In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import numpy as np
import torch.nn.utils.prune as prune
import matplotlib.pyplot as plt
from pruning import Pruning_tool

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print('using', device)

using cpu


## Model :

In [2]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 40)
        self.fc22 = nn.Linear(400, 40)
        self.fc3 = nn.Linear(40, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [3]:
epochs = 10
batch_size = 64
log_interval = 100

mnist_trainset = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
                   batch_size=batch_size, shuffle=True)

mnist_testset = test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
                  batch_size=batch_size, shuffle=True)

model = VAE().to(device)
print(model.parameters)
optimizer = optim.Adam(model.parameters(), lr=7e-4)
step = 0

<bound method Module.parameters of VAE(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=40, bias=True)
  (fc22): Linear(in_features=400, out_features=40, bias=True)
  (fc3): Linear(in_features=40, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=784, bias=True)
)>


In [4]:
def kl_anneal_function(anneal_function, step, k, x0):
    """ Beta update function
        
        Parameters
        ----------
        anneal_function : string
            What type of update (logisitc or linear)
        step : int
            Which step of the training
        k : float
            Coefficient of the logistic function
        x0 : float
            Delay of the logistic function or slope of the linear function
        Returns
        -------
        beta : float
            Weight of the KL divergence in the loss function 
        """
    if anneal_function == 'logistic':
        return float(1/(1+np.exp(-k*(step-x0))))
    elif anneal_function == 'linear':
        return min(1, step/x0)



# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar, beta):
    """ Compute the loss function between recon_x (output of the VAE) 
    and x (input of the VAE)
    """
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = 0.5 * torch.sum(logvar.exp() + mu.pow(2) - 1 - logvar)
    return BCE + beta*KLD

In [5]:
plt_loss = []

for epoch in range(1,2): # epochs + 1):

    model.train()
    train_loss = 0

    for batch_idx, (data, _) in enumerate(mnist_trainset):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)

        beta = kl_anneal_function('linear', step, 1, 10*len(mnist_trainset))
        #print(beta)
        loss = loss_function(recon_batch, data, mu, logvar, beta)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        step += 1

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(mnist_trainset.dataset),
                100. * batch_idx / len(mnist_trainset),
                loss.item() / len(data)))
            plt_loss.append(loss.item() / len(data))  # For ploting loss

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(mnist_trainset.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 549.239624
Train Epoch: 1 [6400/60000 (11%)]	Loss: 182.351730
Train Epoch: 1 [12800/60000 (21%)]	Loss: 134.431595
Train Epoch: 1 [19200/60000 (32%)]	Loss: 125.176575
Train Epoch: 1 [25600/60000 (43%)]	Loss: 110.490181
Train Epoch: 1 [32000/60000 (53%)]	Loss: 103.404694
Train Epoch: 1 [38400/60000 (64%)]	Loss: 95.155647
Train Epoch: 1 [44800/60000 (75%)]	Loss: 91.370392
Train Epoch: 1 [51200/60000 (85%)]	Loss: 97.383255
Train Epoch: 1 [57600/60000 (96%)]	Loss: 91.146004
====> Epoch: 1 Average loss: 123.1770


## Pruning


In [6]:
for name, module in model.named_modules():
    layer = module



#print(module.weight.data.shape)
#print(module.weight.data)
#print(layer.weight)
#print(layer.bias)
print(layer.weight.grad)

tensor([[0.0032, 0.0019, 0.0015,  ..., 0.0024, 0.0009, 0.0005],
        [0.0040, 0.0032, 0.0018,  ..., 0.0037, 0.0013, 0.0009],
        [0.0035, 0.0027, 0.0017,  ..., 0.0034, 0.0019, 0.0010],
        ...,
        [0.0011, 0.0006, 0.0010,  ..., 0.0021, 0.0004, 0.0007],
        [0.0016, 0.0009, 0.0014,  ..., 0.0024, 0.0006, 0.0010],
        [0.0028, 0.0022, 0.0017,  ..., 0.0036, 0.0009, 0.0009]])


In [7]:
from pruning import Pruning

#local pruning
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        pr = Pruning(module,0.3)
        print(module.weight.grad)
        module.register_backward_hook(pr)
        print(pr.mask)

#for batch_idx, (data, _) in enumerate(mnist_trainset):
#        data = data.to(device)
#        optimizer.zero_grad()
#        recon_batch, mu, logvar = model(data)
#        break

prstat = Pruning_tool()
prstat.stats_pruning(model)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[ True, False, False,  ...,  True,  True,  True],
        [ True,  True, False,  ..., False,  True,  True],
        [False, False, False,  ...,  True,  True, False],
        ...,
        [False,  True,  True,  ..., False,  True,  True],
        [ True,  True, False,  ...,  True,  True,  True],
        [False,  True,  True,  ..., False, False,  True]])
tensor([[ 10.9696,   7.4248,  19.4056,  ...,   0.0000,   4.9306,   8.0377],
        [-21.0895, -14.9443, -36.0559,  ...,   0.0000,  -8.9281, -14.8623],
        [ -3.8825,  -5.9315, -10.3392,  ...,   0.0000,  -1.3093,  -8.8553],
        ...,
        [-17.2496, -11.0863, -35.2641,  ...,   0.0000,  -7.1134, -13.0204],
        [-25.7076, -24.6065, -48.9372,  ...,   0.0000, -16.5744, -1

In [9]:
plt_loss = []

for epoch in range(1,2):# epochs + 1):

    model.train()
    train_loss = 0

    for batch_idx, (data, _) in enumerate(mnist_trainset):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)

        beta = kl_anneal_function('linear', step, 1, 10*len(mnist_trainset))
        #print(beta)
        loss = loss_function(recon_batch, data, mu, logvar, beta)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        step += 1

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(mnist_trainset.dataset),
                100. * batch_idx / len(mnist_trainset),
                loss.item() / len(data)))
            plt_loss.append(loss.item() / len(data))  # For ploting loss

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(mnist_trainset.dataset)))

t_features=40, bias=True)
Mask : 
 tensor([[ True, False, False,  ...,  True,  True,  True],
        [ True,  True, False,  ..., False,  True,  True],
        [False, False, False,  ...,  True,  True, False],
        ...,
        [False,  True,  True,  ..., False,  True,  True],
        [ True,  True, False,  ...,  True,  True,  True],
        [False,  True,  True,  ..., False, False,  True]])
Layer : 
 Linear(in_features=784, out_features=400, bias=True)
Mask : 
 tensor([[False,  True,  True,  ...,  True,  True, False],
        [ True,  True, False,  ...,  True,  True, False],
        [False,  True,  True,  ...,  True,  True,  True],
        ...,
        [ True,  True, False,  ...,  True,  True,  True],
        [ True, False, False,  ...,  True, False, False],
        [ True, False, False,  ..., False, False,  True]])
Layer : 
 Linear(in_features=400, out_features=784, bias=True)
Mask : 
 tensor([[False,  True,  True,  ...,  True,  True,  True],
        [ True,  True, False,  ..., Fal